# 逆量子フーリエ変換の実装

位相推定アルゴリズムの中で逆量子フーリエ変換が行われる。　  
シミュレーターを使ってこれを実装する。

In [1]:
from quantum_simulater import *

## やるべきことの確認
1. SWAPゲートで全てのビットを入れ替える。
2. $invR_{m}$ゲートを作る。
3. 第tビットを標的ビットとして、$invR_{n-t}...invR_{3}invR_{2}H$ゲートを作用させる演算を行列表示。($0\leqq t \leqq n-2$)  
第n-1ビットを標的ビットとするときは$H$ゲートのみを作用させる。
4. 全部まとめてn_bitsに対する逆量子フーリエ変換を組む。

# SWAPゲート
スワップゲートを作用させることはは入れ替えたい2ビットa,bに対してCNOTゲートを三回作用させることと同じである。  
ただし1回目の制御ビットをaとすれば2回目の制御ビットはbで3回目はaでなければならない。

In [2]:
# 指定した二つを入れ替える
def SWAP(n_bits, bit_a, bit_b):
    return CNOT(n_bits, bit_a, bit_b) @ CNOT(n_bits, bit_b, bit_a) @ CNOT(n_bits, bit_a, bit_b)

In [3]:
# 全体を入れ替える
def all_SWAP(n_bits):
    AS = I(n_bits)
    for i in range(int(n_bits/2)):
        AS = SWAP(n_bits, i, n_bits-1-i) @ AS
    return AS

In [6]:
qubits = Qubits(5)
qubits.set_bits([0,0,0,1,1])
qubits.apply(all_SWAP(5))
print(qubits)

[1.]|11000>


# $invR_{m}$ゲート
制御回転ゲート$invR_{m}$を作る。引数はビット数, 制御ビット, 標的ビット, 回転させたい角度$-\frac{2\pi}{2^{m}}$の中のmである。  
**n_bitsは2の乗数である。またtargetは $0\leqq target \leqq n-2$ を満たし、mは $2\leqq m \leqq n\_bits-target$ を満たしている。**

In [6]:
def invRm(n_bits, control, target, m):
    U = np.array([[1, 0],
                  [0, math.e**(-2j*math.pi/2**m)]])
    return CU(n_bits, control, target, U)

# tビット目に対する処理
第tビット目を標的ビットにして$invR_{n-t}...invR_{3}invR_{2}H$ゲートを作用させる演算を行列表示。($0\leqq t \leqq n-2$)  
そして制御ビットはt+m-1ビットである。  
ただし第n-1ビット目に対しては$H$ゲートのみを作用させる。

In [7]:
def Opt_t(n_bits, target):
    RH = H(n_bits, target)
    if target == n_bits-1:
        return RH
    else:
        for m in range(2, n_bits-target+1, 1):
            RH = RH @ invRm(n_bits, target+m-1, target, m)
        return RH

# 逆量子フーリエ変換
引数はビット数nのみ

In [8]:
def invQFT(n_bits):
    invQFT = all_SWAP(n_bits)
    for target in range(n_bits-1, -1, -1):
        invQFT = Opt_t(n_bits, target) @ invQFT
    return invQFT

In [9]:
qubits = Qubits(2)
qubits.apply(invQFT(2))
print(qubits)

[0.5+0.j]|00> + [0.5+0.j]|01> + [0.5+0.j]|10> + [0.5+0.j]|11>
